In [19]:
# Airline AI assistant with sqlite database,  Gradio and tool calling

In [20]:
# imports

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import display, Markdown
import json
import sqlite3

In [21]:
# Load environment variables in a file .env

load_dotenv()
openai_key=os.getenv("OPENAI_API_KEY")

# Check the key
if not openai_key:
    print("An API key was not found")
elif openai_key[:8]!="sk-proj-":
    print("An API key was found, but it dosen't start with sk-proj-")
elif openai_key.strip()!= openai_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("An API key was found and looks good")

An API key was found and looks good


In [22]:
# Create an instance of openai
openai = OpenAI()
openai_model = "gpt-4o-mini"

In [23]:
system_message = "You are a helpful assistant for an Airline called FlightAI." \
"Warmly welcome the user to FlightAI assistant." \
"Give short, courtese answers, no more than 1 sentence." \
"Always be accurate. If you don't know the answer, say so."


In [24]:
DB = "prices.db"
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS prices (
        city TEXT PRIMARY KEY,
        price REAL
    )
    """)
    conn.commit()

  

In [25]:
def get_ticket_price(city):
    print (f"Database Tool Called: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT price FROM prices WHERE city = ?", (city.lower(),))
        result = cursor.fetchone()
    return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [26]:
def set_ticket_price(city,price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city,price) VALUES (?,?)  ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(),price, price))
        conn.commit()

In [27]:
# Ticket prices
ticket_prices ={
    "london": "799",
    "paris":"899",
    "tokyo":"1400",
    "berlin":"499"

}
for city,price in ticket_prices.items():
    set_ticket_price(city,price)


In [28]:
get_ticket_price("bgt")

Database Tool Called: Getting price for bgt


'No price data available for this city'

In [29]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                    "role": "tool",
                    "content": price_details,
                    "tool_call_id": tool_call.id
                })
    return responses

In [30]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [31]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [32]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=openai_model, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        tool_responses = handle_tool_call(message)
        
        # Add the assistant's message with tool calls
        messages.append(message)
        
        # Add all tool responses
        for tool_response in tool_responses:
            messages.append(tool_response)
        
        # Get final response from the model after tool responses
        response = openai.chat.completions.create(model=openai_model, messages=messages)
    
    return response.choices[0].message.content

Tools

In [33]:
# get_ticket_price("abuja")

In [34]:
# Gradio implementation
gr.ChatInterface(fn=chat, title="FlightAI Assistant").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


Database Tool Called: Getting price for London
